In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indonesian_number_normalizer import create_normalizer
from tqdm import tqdm

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
normalizer = create_normalizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini.csv')
df

,content,title
0,Ikatan Dokter Indonesia (IDI) menanggapi fenom...,IDI soal Ida Dayak: Pengobatan Alternatif Haru...
1,"JAKARTA, KOMPAS.com - Penjabat (Pj) Gubernur ...","HUT Ke-79 RI di IKN, Heru Budi Sebut Infrastru..."
2,Konflik antara dan pernah ramai dibicarakan. S...,"Persija Harus Bayar Marko Simic Total Rp 7 M, ..."
3,"Jakarta, CNBC Indonesia - PT Telkom Indonesia ...","Jelang Kawin Dengan Telkomsel, Nilai IndiHome ..."
4,"TEMPO.CO, Jakarta - Mantan Gubernur DKI Jakart...",Anies Baswedan Mau Buat Partai Politik setelah...
...,...,...
5995,"TEMPO.CO, Jakarta - Bursa Pilkada Jakarta 2024...",Kata Peneliti Soal Peluang Anies Baswedan di P...
5996,"Serangan militer Israel ke wilayah Gaza, Pales...",Gugur Wali Kota Gaza Digempur Israel kala Tinj...
5997,"TEMPO.CO, Jakarta - Calon presiden Ganjar Pran...","Ganjar Soroti Banyak Pejabat Tak Isi LHKPN, Ap..."
5998,"JAKARTA, KOMPAS.com - Partai Gerindra sangat ...","Momen Muzani ""Goda"" Jokowi di Rapimnas Gerindr..."


In [5]:
df = df.drop_duplicates(subset=['content'])
print(f"Dataset setelah dibersihkan memiliki {len(df)} baris")

Dataset setelah dibersihkan memiliki 6000 baris
